<a href="https://colab.research.google.com/github/cdbolivarz/Accidentality_Predictions_Medellin/blob/master/Limpieza__y_formato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza y formato de datos
En este documento, se realiza el preprocesamiento de los datos obtenidos de los documentos de incidentes georeferenciados.

Librerias

In [1]:
import pandas as pd
import glob
import numpy as np
import re

### Lectura de datos

In [2]:
all_files = glob.glob("dataset/I*.csv")

li = []

for filename in all_files:

    frame = pd.read_csv(filename, index_col=None, header=0, low_memory=False)
    li.append(frame)

df = pd.concat(li, axis=0, ignore_index=True)
df = df.sort_values(by=["FECHA"])
df = df.reset_index()
df = df.drop(['index'],axis=1)
df.head()

,X,Y,OBJECTID,RADICADO,FECHA,HORA,DIA,PERIODO,CLASE,DIRECCION,DIRECCION_ENC,CBML,TIPO_GEOCOD,GRAVEDAD,BARRIO,COMUNA,DISENO,DIA_NOMBRE,MES,MES_NOMBRE,X_MAGNAMED,Y_MAGNAMED,LONGITUD,LATITUD
0,-75.703816,6.221806,211342,1424116,2014/01/01 00:00:00+00,05:30:00,1,2014,Atropello,Con,000 00000,NaN,No Ubicada,HERIDO,NaN,NaN,Lote o Predio,MIÉRCOLES,1,NaN,820000.00,1180000.00,-75.703816,6.221806
1,-75.579329,6.209160,211341,1424110,2014/01/01 00:00:00+00,04:20:00,1,2014,Choque,CR 50 CL 4,CR 050 004 000 00000,1507,Malla vial,HERIDO,Campo Amor,Guayabal,Tramo de via,MIÉRCOLES,1,NaN,833779.95,1178599.23,-75.579329,6.209160
2,-75.565164,6.242722,211340,1424107,2014/01/01 00:00:00+00,09:10:00,1,2014,Choque,CL 44 CR 43,CL 044 043 000 00000,1013,Malla vial,HERIDO,Barrio Colón,La Candelaria,Interseccion,MIÉRCOLES,1,NaN,835348.01,1182311.63,-75.565164,6.242722
3,-75.555400,6.252163,211339,1424101,2014/01/01 00:00:00+00,08:50:00,1,2014,Choque,CR 39 CL 60,CR 039 060 000 00000,0803,Malla vial,HERIDO,San Miguel,Villa Hermosa,Tramo de via,MIÉRCOLES,1,NaN,836428.72,1183355.92,-75.555400,6.252163
4,-75.564683,6.249282,211338,1424096,2014/01/01 00:00:00+00,08:00:00,1,2014,Otro,CL 52 CR 46,CL 052 046 000 00000,1019,Malla vial,HERIDO,La Candelaria,La Candelaria,Tramo de via,MIÉRCOLES,1,NaN,835401.20,1183037.23,-75.564683,6.249282


Que columnas tienen valores nulos

In [3]:
df.isna().any()

X                False
Y                False
OBJECTID         False
RADICADO          True
FECHA            False
HORA             False
DIA              False
PERIODO          False
CLASE             True
DIRECCION        False
DIRECCION_ENC     True
CBML              True
TIPO_GEOCOD      False
GRAVEDAD         False
BARRIO            True
COMUNA            True
DISENO            True
DIA_NOMBRE       False
MES              False
MES_NOMBRE        True
X_MAGNAMED       False
Y_MAGNAMED       False
LONGITUD         False
LATITUD          False
dtype: bool

Formato datos

In [4]:
#Eliminar filas con NA y columnas que no consideramos importantes para el modelo.
#Asignar mismos nombres a los tipos de accidentes (CLASE)
def format(df):
  df = df.drop(['X', 'Y', 'OBJECTID', 'RADICADO', 'DIRECCION', 'DIRECCION_ENC',
              'CBML', 'TIPO_GEOCOD', 'DISENO', 'MES_NOMBRE', 'X_MAGNAMED',
              'Y_MAGNAMED'], axis=1)
  df = df.dropna(subset=['CLASE', 'BARRIO', 'COMUNA'])

  df["CLASE"].replace({"Caída de Ocupante": "Caida Ocupante",
                       "Caida de Ocupante": "Caida Ocupante",
                       "Caída Ocupante": "Caida Ocupante",
                       "Choque ": "Choque"}, inplace=True)

  return df

df = format(df)
df.columns = df.columns.str.lower()
df.head()

,fecha,hora,dia,periodo,clase,gravedad,barrio,comuna,dia_nombre,mes,longitud,latitud
1,2014/01/01 00:00:00+00,04:20:00,1,2014,Choque,HERIDO,Campo Amor,Guayabal,MIÉRCOLES,1,-75.579329,6.209160
2,2014/01/01 00:00:00+00,09:10:00,1,2014,Choque,HERIDO,Barrio Colón,La Candelaria,MIÉRCOLES,1,-75.565164,6.242722
3,2014/01/01 00:00:00+00,08:50:00,1,2014,Choque,HERIDO,San Miguel,Villa Hermosa,MIÉRCOLES,1,-75.555400,6.252163
4,2014/01/01 00:00:00+00,08:00:00,1,2014,Otro,HERIDO,La Candelaria,La Candelaria,MIÉRCOLES,1,-75.564683,6.249282
5,2014/01/01 00:00:00+00,10:00:00,1,2014,Choque,HERIDO,La Milagrosa,Buenos Aires,MIÉRCOLES,1,-75.557573,6.235498


Filas con errores ortograficos en algunas columnas

In [5]:
df['comuna'].drop_duplicates()

1              Guayabal
2         La Candelaria
3         Villa Hermosa
5          Buenos Aires
7               Popular
              ...      
184133        Altavista
184435         Girardot
184440     Barrio Colón
184471       Campo Amor
184550        Andalucía
Name: comuna, Length: 84, dtype: object

In [6]:
df['barrio'].drop_duplicates()

1               Campo Amor
2             Barrio Colón
3               San Miguel
4            La Candelaria
5             La Milagrosa
                ...       
205841            El Uvito
207239                7001
207307                9004
213772    Volcana Guayabal
220429             El Plan
Name: barrio, Length: 339, dtype: object

In [7]:
df.head()

,fecha,hora,dia,periodo,clase,gravedad,barrio,comuna,dia_nombre,mes,longitud,latitud
1,2014/01/01 00:00:00+00,04:20:00,1,2014,Choque,HERIDO,Campo Amor,Guayabal,MIÉRCOLES,1,-75.579329,6.209160
2,2014/01/01 00:00:00+00,09:10:00,1,2014,Choque,HERIDO,Barrio Colón,La Candelaria,MIÉRCOLES,1,-75.565164,6.242722
3,2014/01/01 00:00:00+00,08:50:00,1,2014,Choque,HERIDO,San Miguel,Villa Hermosa,MIÉRCOLES,1,-75.555400,6.252163
4,2014/01/01 00:00:00+00,08:00:00,1,2014,Otro,HERIDO,La Candelaria,La Candelaria,MIÉRCOLES,1,-75.564683,6.249282
5,2014/01/01 00:00:00+00,10:00:00,1,2014,Choque,HERIDO,La Milagrosa,Buenos Aires,MIÉRCOLES,1,-75.557573,6.235498


In [9]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
len(df['barrio'].drop_duplicates())

339

Todos las columnas de *clase, gravedad, barrio, comuna, dia_nombre* se pasaron a miniuscula para evitar posibles inconveniesntes futuros

In [10]:
dfcopy = df.copy()
dfcopy['clase'] = dfcopy['clase'].str.lower()
dfcopy['gravedad'] = dfcopy['gravedad'].str.lower()
dfcopy['barrio'] = dfcopy['barrio'].str.lower()
dfcopy['comuna'] = dfcopy['comuna'].str.lower()
dfcopy['dia_nombre'] = dfcopy['dia_nombre'].str.lower()

## Barrios


Se eliminan las filas donde el barrio sale como un número

In [11]:
for barrio in dfcopy['barrio']:
    try:
        int(barrio)
        dfcopy = dfcopy.replace({'barrio': barrio}, np.nan)
    except:
        pass
dfcopy= dfcopy.dropna(subset=['barrio'])

Para corregir los barrios donde salía **no. #**  para que salga **no.#**

In [12]:
def error_no(row):
    barrio = row['barrio'].strip()
    b = bool(re.search(r'(no)\. \d', barrio))
    if (b):
        b = barrio.split()
        barrio = ' '.join(b[0:-2])
        barrio += ' '+b[-2]+b[-1]
    row['barrio'] = barrio
    return row
dfcopy = dfcopy.apply(error_no,axis=1)

In [13]:
# Barrios con problemas
# en Medellin se supone hay 275 barrio (segun wikipedia)

# hay varios suburbanos que me parecen raros
# el barrio 'u.p.b'
# creo que todos los barrios que se llaman universidad algo estan mal (el de falcultad de minas existe)


In [14]:
dfcopy = dfcopy.replace({'barrio':{'barrio de jesús':'barrios de jesús',
                                   'berlin':'berlín',
                                   'inst':np.nan,
                                   'loma de los bernal':'la loma de los bernal',
                                   'villa liliam':'villa lilliam'}})

In [15]:
barrios = dfcopy['barrio'].sort_values().unique()
barrios

array(['aguas frias', 'aldea pablo vi', 'alejandro echavarría',
       'alejandría', 'alfonso lópez', 'altamira', 'altavista',
       'altavista sector central', 'altos del poblado', 'andalucía',
       'antonio nariño', 'aranjuez', 'area de expansion altavista',
       'area de expansion belen rincon', 'asomadera no.1',
       'asomadera no.2', 'asomadera no.3', 'astorga', 'auc1', 'auc2',
       'aures no.1', 'aures no.2', 'b. cerro  el volador',
       'b. cerro el volador', 'barrio caicedo', 'barrio colombia',
       'barrio colón', 'barrio cristóbal', 'barrios de jesús',
       'batallón girardot', 'belalcázar', 'belencito', 'bello horizonte',
       'belén', 'berlín', 'bermejal-los alamos', 'betania', 'blanquizal',
       'bolivariana', 'bomboná no.1', 'bomboná no.2',
       'bosques de san pablo', 'boston', 'boyacá', 'brasilia',
       'buenos aires', 'buga patio bonito',
       'cabecera san antonio de prado', 'cabecera urbana san cristobal',
       'calasanz', 'calasanz parte a

In [16]:
len(barrios)

317

## Comunas

In [17]:
# 1	Comuna 1 - Popular
# 2	Comuna 2 - Santa Cruz
# 3	Comuna 3 - Manrique
# 4	Comuna 4 - Aranjuez
# 5	Comuna 5 - Castilla
# 6	Comuna 6 - Doce de Octubre
# 7	Comuna 7 - Robledo
# 8	Comuna 8 - Villa Hermosa
# 9	Comuna 9 - Buenos Aires
# 10	Comuna 10 - La Candelaria
# 11	Comuna 11 - Laureles-Estadio
# 12	Comuna 12 - La América
# 13	Comuna 13 - San Javier
# 14	Comuna 14 - El Poblado
# 15	Comuna 15 - Guayabal
# 16	Comuna 16 - Belén

In [18]:
dfcopy = dfcopy.replace({'comuna':{ 'alejandro echavarría': 'buenos aires',
                                    'alfonso lópez':'castilla',
                                    'altavista':'belén',
                                    'andalucía':'santa cruz',
                                    'antonio nariño':'san javier',
                                    'au':np.nan,
                                    'barrio colón':'la candelaria',
                                    'bolivariana':'laureles-estadio',
                                    'boston':'la candelaria',
                                    'cabecera san antonio de prado':'corregimiento san antorio de prado',
                                    'calasanz':'la américa',
                                    'calle nueva':'la candelaria',
                                    'campo amor':'guayabal',
                                    'campo valdés no. 1':'aranjuez',
                                    'campo valdés no. 2':'manrique',
                                    'caribe':'castilla',
                                    'cerro nutibara':'belén',
                                    'corazón de jesús':'la candelaria',
                                    'cristo rey':'guayabal',
                                    'el chagualo':'la candelaria',
                                    'el nogal-los almendros':'belén',
                                    'el raizal':'manrique',
                                    'estación villa':'la candelaria',
                                    'florida nueva':'laureles-estadio',
                                    'fátima':'belén',
                                    'girardot':'castilla',
                                    'guayaquil':'la candelaria',
                                    'héctor abad gómez':'castilla'}})

In [19]:
dfcopy = dfcopy.replace({'comuna':{'in':np.nan, 
                                'jesús nazareno':'la candelaria',
                                'la alpujarra':np.nan,
                                'la colina':'guayabal',
                                'la floresta':'la américa',
                                'la rosa':'santa cruz',
                                'las esmeraldas':'aranjuez',
                                'las granjas':'manrique',
                                'las palmas':'la candelaria',
                                'las playas':'belén',
                                'laureles':'laureles-estadio',
                                'laureles estadio':'laureles-estadio', 
                                'los conquistadores':'laureles-estadio',
                                'los mangos':'villa hermosa',
                                'manila':'el poblado',
                                'miranda':'aranjuez',
                                'moravia':'aranjuez',
                                'naranjal':'laureles-estadio',
                                'oleoducto':np.nan,
                                'parque juan pablo ii':'buenos aires',
                                'patio bonito':'el poblado',
                                'perpetuo socorro':'la candelaria',
                                'rosales':'belén',
                                'san pedro':'aranjuez',
                                'santa fé':'guayabal',
                                'santa inés':'manrique',
                                'santa maría de los ángeles':'el poblado',
                                'simón bolívar':'la américa',
                                'sn':np.nan,
                                'suramericana':'laureles-estadio',
                                'toscana':'castilla',
                                'universidad de antioquia':np.nan,
                                'veinte de julio':'san javier',
                                'villa carlota':'el poblado',
                                'villa guadalupe':'popular'}})

In [20]:
comunas = dfcopy['comuna'].sort_values().unique()
comunas

array(['aranjuez', 'belén', 'buenos aires', 'castilla',
       'corregimiento de altavista',
       'corregimiento de san antonio de prado',
       'corregimiento de san cristóbal',
       'corregimiento de san sebastián de palmitas',
       'corregimiento de santa elena',
       'corregimiento san antorio de prado', 'doce de octubre',
       'el poblado', 'guayabal', 'la américa', 'la candelaria',
       'laureles-estadio', 'manrique', 'popular', 'robledo', 'san javier',
       'santa cruz', 'villa hermosa', nan], dtype=object)

## Fechas

Primero se cambia el timpo de la comuna *fecha* por el tipo **datetime**

In [21]:
fechas = df['fecha']
dfcopy['fecha'] = pd.to_datetime(fechas)

Luego se crea la columna *dia_anno* para saber a que dia del año pertenece cada fila

In [22]:
def dia_anno(row):
    row['dia_anno'] = row['fecha'].dayofyear
    return row
dfcopy = dfcopy.apply(dia_anno, axis=1)
